## Initialize Environment

In [1]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import logging
from shapely import MultiPolygon
from shapely.wkt import dumps as wkt_dumps # Use Shapely directly for WKT

from IPython.display import display

# Configure logging to display messages in the notebook output
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", force=True
)

pd.set_option("display.max_columns", None)

## Load Data

In [2]:
fatal_crashes_df = pd.read_excel(
    "data/Project/bitre_fatal_crashes_dec2024.xlsx",
    sheet_name="BITRE_Fatal_Crash",
    skiprows=4,
    na_values=["", "-9"],
)
# Wrong header name fix
fatal_crashes_df = fatal_crashes_df.rename(
    columns={"Bus \nInvolvement": "Bus Involvement", "Time of Day": "Time of day"},
)
fatalities_df = pd.read_excel(
    "data/Project/bitre_fatalities_dec2024.xlsx",
    sheet_name="BITRE_Fatality",
    skiprows=4,
    na_values=["", "-9"],
)

In [3]:
pop_LGA_df = pd.read_excel(
    "data/Project/Population estimates by LGA, Significant Urban Area, Remoteness Area, Commonwealth Electoral Division and State Electoral Division, 2001 to 2023.xlsx",
    sheet_name="Table 1",
    header=None,
    skiprows=7,
    skipfooter=2,
    usecols="B:Y",
)
pop_remoteness_df = pd.read_excel(
    "data/Project/Population estimates by LGA, Significant Urban Area, Remoteness Area, Commonwealth Electoral Division and State Electoral Division, 2001 to 2023.xlsx",
    sheet_name="Table 3",
    header=None,
    skiprows=7,
    skipfooter=7,
    usecols="A:Y",
)

In [109]:
dwellings_df = pd.read_csv(
    "data/Project/LGA (count of dwellings).csv",
    skiprows=11,        # Skip the metadata header lines
    skipfooter=9,       # Skip the metadata footer lines and Total row
    usecols=range(2),   # Only keep the first two columns
    engine="python",    # Use Python engine to handle the footer
    header=None,        # No header in the CSV
)

### Deal with unknown values

In [5]:
# Deal with int value
fatal_crashes_df.replace(-9, pd.NA, inplace=True)
fatalities_df.replace(-9, pd.NA, inplace=True)

## Merge Crash and Fatality Data

In [6]:
fact_df = pd.merge(
    fatalities_df, fatal_crashes_df, on="Crash ID", suffixes=("_victim", "_crash")
)

## Data Cleaning

In [7]:
# Identify rows to be deleted (where Crash ID length ≠ 8)
rows_to_delete = fact_df[fact_df["Crash ID"].astype(str).str.len() != 8]

# Print only the Crash IDs that will be deleted
print("Crash IDs to be deleted (length ≠ 8):")
print(rows_to_delete["Crash ID"].to_string(index=False))

print(f"\nTotal rows to be deleted: {len(rows_to_delete)}")

# Execute deletion (keep only rows where Crash ID length equals 8)
fact_df = fact_df[fact_df["Crash ID"].astype(str).str.len() == 8]

# Verification
print(f"\nRemaining rows: {len(fact_df)}")
print("Crash ID length distribution in remaining data:")
print(fact_df["Crash ID"].astype(str).str.len().value_counts())

Crash IDs to be deleted (length ≠ 8):
201850049
201850039
201850059
201750029
201750019
201650079
201650019
201550039
201550099
201550059

Total rows to be deleted: 10

Remaining rows: 56864
Crash ID length distribution in remaining data:
Crash ID
8    56864
Name: count, dtype: int64


In [8]:
# Identify rows with empty Time field
empty_time_rows = fact_df[fact_df["Time_victim"].isna()]

# Print only the Crash IDs of rows with empty Time (assuming Crash ID exists)
if not empty_time_rows.empty:
    print("Crash IDs with empty Time field to be deleted:")
    print(empty_time_rows["Crash ID"].to_string(index=False))
else:
    print("No rows with empty Time field found")

print(f"\nTotal rows to be deleted: {len(empty_time_rows)}")

# Execute deletion (keep only rows with non-empty Time)
fact_df = fact_df[fact_df["Time_victim"].notna()]

# Verification
print(f"\nRemaining rows: {len(fact_df)}")
print(
    f"Number of null Time values after deletion: {fact_df['Time_victim'].isna().sum()}"
)

Crash IDs with empty Time field to be deleted:
20247030
20247040
20247033
20127002
20087036
20087041
20077047
20077047
20057036
20057045
20057045
20052184
20057028
20042247
20036028
20022015
20012201
20012064
20012134
20012165
20002042
19982058
19988013
19988020
19986046
19962220
19958014
19958004
19958001
19958001
19952079
19952356
19958003
19946050
19935187
19935187
19935161
19935123
19935191
19912147
19915105
19902391
19895029

Total rows to be deleted: 43

Remaining rows: 56821
Number of null Time values after deletion: 0


### Special Fix

In [9]:
# Set missing age_group
fact_df.loc[fact_df["Crash ID"] == 20225110, "Age Group"] = "17_to_25"

In [10]:
# Fix missing Time of day
fact_df.loc[fact_df["Crash ID"] == 20225066, "Time of day_victim"] = "Day"

## Create Dimension Tables (Use fatalities_df as main table)

In [11]:
state_mapping = {
    "New South Wales": "NSW",
    "Victoria": "Vic",
    "Queensland": "Qld", # Make sure abbreviation matches your data
    "Western Australia": "WA",
    "South Australia": "SA",
    "Tasmania": "Tas",
    "Northern Territory": "NT",
    "Australian Capital Territory": "ACT",
    # Add others if necessary (e.g., Other Territories)
}

### dim_date

In [17]:
dim_date_df = fact_df[
    [
        "Year_victim",
        "Month_victim",
        "Christmas Period_victim",
        "Easter Period_victim",
    ]
].copy()

dim_date_df.rename(
    columns={
        "Year_victim": "year",
        "Month_victim": "month",
        "Christmas Period_victim": "christmas_period",
        "Easter Period_victim": "easter_period",
    },
    inplace=True,
)

# Derive additional columns
month_map = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December",
}
dim_date_df["month_name"] = dim_date_df["month"].map(month_map)
dim_date_df["quarter"] = ((dim_date_df["month"] - 1) // 3) + 1
dim_date_df["month_year"] = (
    dim_date_df["year"].astype(str) + "-" + dim_date_df["month"].astype(str).str.zfill(2)
)

# Select columns in the final table order and drop duplicates
dim_date_df = (
    dim_date_df[
        [
            "year",
            "month",
            "month_name",
            "quarter",
            "month_year",
            "christmas_period",
            "easter_period",
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

In [18]:
dim_date_df.to_csv("output/dim_date.csv", index=False)

### dim_time_of_day

In [19]:
dim_timeofday_df = fact_df[["Time_victim", "Time of day_victim"]].copy()

dim_timeofday_df.rename(
    columns={
        "Time_victim": "time",
        "Time of day_victim": "time_of_day_category",
    },
    inplace=True,
)

# Convert to time objects to handle potential variations and extract components
# Handle potential errors during conversion if format is inconsistent
try:
    dim_timeofday_df["time"] = pd.to_datetime(
        dim_timeofday_df["time"], format="%H:%M:%S", errors="coerce"
    ).dt.time
except ValueError:
    print(
        "Warning: Could not parse all time values. Check format consistency (HH:MM:SS). Using NaT for unparseable times."
    )
    dim_timeofday_df["time"] = pd.to_datetime(
        dim_timeofday_df["time"], errors="raise"  # Raise error if format is inconsistent
    ).dt.time  # Try without format

dim_timeofday_df["hour_24"] = pd.to_datetime(
    dim_timeofday_df["time"].astype(str), format="%H:%M:%S"
).dt.hour.astype('Int64')


# Select final columns and drop duplicates
dim_timeofday_df = (
    dim_timeofday_df[["time_of_day_category", "hour_24", "time"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

In [20]:
dim_timeofday_df.to_csv("output/dim_timeofday.csv", index=False)

### dim_road_user

In [21]:
dim_roaduser_df = fact_df[["Road User", "Gender", "Age", "Age Group"]].copy()
dim_roaduser_df.rename(
    columns={
        "Road User": "road_user_type",
        "Gender": "gender",
        "Age": "age",
        "Age Group": "age_group",
    },
    inplace=True,
)

dim_roaduser_df["gender"] = dim_roaduser_df["gender"].fillna("Unknown")
dim_roaduser_df["age_group"] = dim_roaduser_df["age_group"].fillna("Unknown")

dim_roaduser_df = (
    dim_roaduser_df[["road_user_type", "gender", "age", "age_group"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

In [22]:
dim_roaduser_df.to_csv("output/dim_roaduser.csv", index=False)

### dim_crash

In [23]:
dim_crash_df = fact_df[
    [
        "Crash ID",
        "Crash Type_victim",
        "Bus Involvement_victim",
        "Heavy Rigid Truck Involvement_victim",
        "Articulated Truck Involvement_victim",
        "Speed Limit_victim",
        "National Road Type_victim",
    ]
].copy()

dim_crash_df.rename(
    columns={
        "Crash ID": "crash_id",
        "Crash Type_victim": "crash_type",
        "Bus Involvement_victim": "bus_involvement",
        "Heavy Rigid Truck Involvement_victim": "heavy_rigid_truck_involvement",
        "Articulated Truck Involvement_victim": "articulated_truck_involvement",
        "Speed Limit_victim": "speed_limit",
        "National Road Type_victim": "national_road_type",
    },
    inplace=True,
)

columns_to_fill = [
    "crash_type",
    "bus_involvement",
    "heavy_rigid_truck_involvement",
    "articulated_truck_involvement",
    "speed_limit",
    "national_road_type",
]
dim_crash_df[columns_to_fill] = dim_crash_df[columns_to_fill].fillna("Unknown")
# Keep only the first occurrence for each Crash_ID to define the dimension attributes
dim_crash_df = dim_crash_df.drop_duplicates(subset=["crash_id"]).reset_index(drop=True)

In [24]:
dim_crash_df = dim_crash_df[
    [
        "crash_id",
        "crash_type",
        "bus_involvement",
        "heavy_rigid_truck_involvement",
        "articulated_truck_involvement",
        "speed_limit",
        "national_road_type",
    ]
]
dim_crash_df.to_csv("output/dim_crash.csv", index=False)

### dim_location

In [25]:
GEOJSON_PATH = "data/Project/Geojson files/"  # Base path to GeoJSONs
OUTPUT_CSV_FILE = "output/dim_location.csv"  # Output CSV file
LGA_TARGET_SRID = 4283
SA4_STATE_TARGET_SRID = 7844

#### Step 1: Extract Unique Location Attributes

This cell extracts the necessary attribute columns from fact_df, renames them for consistency with the target table, and finds the unique combinations.

In [39]:
logging.info("Extracting unique location attributes from fact_df...")
location_attributes_df = fact_df[
    [
        "State_victim",
        "SA4 Name 2021_victim",
        "National LGA Name 2021_victim",
        "National Remoteness Areas_victim",
    ]
].copy()

# Rename columns
location_attributes_df.rename(
    columns={
        "State_victim": "state",
        "SA4 Name 2021_victim": "sa4_name_2021",
        "National LGA Name 2021_victim": "lga_name_2021",
        "National Remoteness Areas_victim": "remoteness_area",
    },
    inplace=True,
)

location_attributes_df.fillna("Unknown", inplace=True)
# Drop exact duplicates
unique_locations_df = location_attributes_df.drop_duplicates().reset_index(drop=True)
logging.info(
    f"Found {len(unique_locations_df)} unique location attribute combinations."
)

logging.info("Displaying unique location attributes (head):")
display(unique_locations_df.head())

2025-04-09 22:46:36,383 - INFO - Extracting unique location attributes from fact_df...
2025-04-09 22:46:36,427 - INFO - Found 802 unique location attribute combinations.
2025-04-09 22:46:36,427 - INFO - Displaying unique location attributes (head):


,state,sa4_name_2021,lga_name_2021,remoteness_area
0,NSW,Riverina,Wagga Wagga,Inner Regional Australia
1,NSW,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Inner Regional Australia
2,Tas,Launceston and North East,Northern Midlands,Inner Regional Australia
3,NSW,New England and North West,Armidale Regional,Outer Regional Australia
4,Vic,Unknown,Unknown,Unknown


#### Step 2: Load Geometries

Define a helper function to load GeoJSON files for each administrative level and then call it.

In [40]:
def load_geometries(geojson_filename, name_property, expected_level, crs=f"EPSG:7844", base_path=GEOJSON_PATH):
    """
    Loads geometries from a SINGLE specified GeoJSON file.

    Args:
        geojson_filename (str): The specific filename (e.g., 'sa4_boundaries_2021.geojson').
        name_property (str): The property name in the GeoJSON containing the location name
        (e.g., 'LGA_NAME21', 'SA4_NAME21', 'STATE_NAME').
        expected_level (str): The level name (e.g., 'LGA', 'SA4', 'State') for logging.
        crs (str): The target Coordinate Reference System.
        base_path (str): Optional base directory where the GeoJSON file is located.

    Returns:
        geopandas.GeoDataFrame: GeoDataFrame for the file, or None if processing fails.
    """
    full_path = os.path.join(base_path, geojson_filename)
    logging.info(f"Attempting to load {expected_level} geometries from: {full_path} using name property: {name_property}...")

    if not os.path.exists(full_path):
        logging.error(f"File not found: {full_path}")
        return None

    try:
        gdf = gpd.read_file(full_path)

        # --- Basic Validation ---
        if 'geometry' not in gdf.columns:
            logging.error(f"File {geojson_filename}: No 'geometry' column found.")
            return None
        if name_property not in gdf.columns:
            logging.error(f"File {geojson_filename}: Required property '{name_property}' not found in columns: {gdf.columns.tolist()}")
            return None
        if gdf.empty or gdf['geometry'].isnull().all():
            logging.warning(f"File {geojson_filename}: Empty or contains no valid geometries.")
            return None
        # Remove rows with null geometry
        gdf = gdf[gdf['geometry'].notna()]
        if gdf.empty:
            logging.warning(f"File {geojson_filename}: No valid geometries after removing nulls.")
            return None

        # --- CRS Handling ---
        # Use gdf.crs.srs to get the string representation like 'EPSG:xxxx'
        current_crs_str = gdf.crs.srs.upper() if gdf.crs else None
        target_crs_str = crs.upper()

        if current_crs_str and current_crs_str != target_crs_str:
            logging.info(f"Reprojecting {geojson_filename} from {current_crs_str} to {target_crs_str}...")
            gdf = gdf.to_crs(target_crs_str)
        elif not current_crs_str:
            # Attempt to guess common CRS if missing, otherwise error
            try:
                logging.warning(f"File {geojson_filename} has no CRS defined. Assuming it's EPSG:4326 (WGS84) and reprojecting to {target_crs_str}. **VERIFY THIS ASSUMPTION!**")
                gdf.set_crs(f"EPSG:4326", inplace=True) # Common default, adjust if wrong!
                gdf = gdf.to_crs(target_crs_str)
            except Exception as crs_err:
                logging.error(f"Failed to set/reproject missing CRS for {geojson_filename}. Error: {crs_err}")
                return None
        # Ensure target SRID is set explicitly in the GeoDataFrame's CRS attribute
        gdf = gdf.set_crs(target_crs_str, allow_override=True)

        # Keep only the name and geometry, standardize name column
        gdf_processed = gdf[[name_property, 'geometry']].rename(columns={name_property: 'name'})

        # Convert Polygons to MultiPolygons if needed by DB schema
        gdf_processed['geometry'] = gdf_processed['geometry'].simplify(tolerance=0.0001, preserve_topology=True)
        gdf_processed['geometry'] = gdf_processed.geometry.apply(lambda geom: MultiPolygon([geom]) if geom.geom_type == 'Polygon' else geom)

        def get_vertex_count(geom):
            if geom.is_empty:
                return 0

            if geom.geom_type == 'MultiPolygon':
                count = 0
                for poly in geom.geoms:
                    count += len(poly.exterior.coords)
                    count += sum(len(interior.coords) for interior in poly.interiors)
                return count
            else:
                return 0
        print("The number of point:\n", gdf_processed['geometry'].apply(get_vertex_count))


        # Drop duplicates based on the name property within this file
        gdf_processed = gdf_processed.drop_duplicates(subset=['name'])
        logging.info(f"Loaded {len(gdf_processed)} unique {expected_level} geometries from {geojson_filename}.")
        return gdf_processed

    except Exception as e:
        logging.error(f"Error reading or processing file {geojson_filename}: {e}")
        return None

In [41]:
lga_geom_gdf = load_geometries(
    geojson_filename="LGA_2021_AUST_GDA94.geojson",
    name_property="LGA_NAME21",
    expected_level="LGA",
    crs="EPSG:4283",  # GDA94
)
sa4_geom_gdf = load_geometries(
    geojson_filename="SA4_2021_AUST_GDA2020.geojson",
    name_property="SA4_NAME21",
    expected_level="SA4",
    crs="EPSG:7844",  # GDA2020
)
state_geom_gdf = load_geometries(
    geojson_filename="STE_2021_AUST_GDA2020.geojson",
    name_property="STE_NAME21",
    expected_level="State",
    crs="EPSG:7844",  # GDA2020
)

# Display heads of loaded geodataframes (optional check)
if lga_geom_gdf is not None:
    display(lga_geom_gdf.head())
if sa4_geom_gdf is not None:
    display(sa4_geom_gdf.head())
if state_geom_gdf is not None:
    display(state_geom_gdf.head())

2025-04-09 22:46:41,632 - INFO - Attempting to load LGA geometries from: data/Project/Geojson files/LGA_2021_AUST_GDA94.geojson using name property: LGA_NAME21...
/var/folders/zx/glnr12gn0pg2611688cv8bx00000gn/T/ipykernel_84126/2026990845.py:37: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  gdf = gdf[gdf['geometry'].notna()]
/opt/homebrew/Caskroom/miniconda/base/envs/DataWarehousing/lib/python3.12/site-packages/shapely/constructive.py:863: RuntimeWarning: invalid value encountered in simplify_preserve_topology
  return lib.simplify_preserve_topo

The number of point:
 0       897
1      3510
2      1179
3      5076
4      2966
       ... 
561       0
562    1507
563       0
564       0
565       0
Name: geometry, Length: 566, dtype: int64


/var/folders/zx/glnr12gn0pg2611688cv8bx00000gn/T/ipykernel_84126/2026990845.py:37: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  gdf = gdf[gdf['geometry'].notna()]
/opt/homebrew/Caskroom/miniconda/base/envs/DataWarehousing/lib/python3.12/site-packages/shapely/constructive.py:863: RuntimeWarning: invalid value encountered in simplify_preserve_topology
  return lib.simplify_preserve_topology(geometry, tolerance, **kwargs)
2025-04-09 22:47:00,462 - INFO - Loaded 108 unique SA4 geometries from SA4_2021_AUST_GDA2020.geojson.
2025-04-09 22:47:00,467 -

The number of point:
 0      15236
1       4144
2      12005
3       4139
4       9555
       ...  
103        0
104     3580
105        0
106        0
107        0
Name: geometry, Length: 108, dtype: int64


/var/folders/zx/glnr12gn0pg2611688cv8bx00000gn/T/ipykernel_84126/2026990845.py:37: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  gdf = gdf[gdf['geometry'].notna()]
/opt/homebrew/Caskroom/miniconda/base/envs/DataWarehousing/lib/python3.12/site-packages/shapely/constructive.py:863: RuntimeWarning: invalid value encountered in simplify_preserve_topology
  return lib.simplify_preserve_topology(geometry, tolerance, **kwargs)
2025-04-09 22:47:06,866 - INFO - Loaded 10 unique State geometries from STE_2021_AUST_GDA2020.geojson.


The number of point:
 0     55258
1     33212
2     74404
3     38256
4    122392
5     94825
6     41551
7      1398
8      3577
9         0
Name: geometry, dtype: int64


,name,geometry
0,Albury,"MULTIPOLYGON (((146.86565 -36.07293, 146.86481..."
1,Armidale Regional,"MULTIPOLYGON (((151.32425 -30.26923, 151.32295..."
2,Ballina,"MULTIPOLYGON (((153.57106 -28.87382, 153.57178..."
3,Balranald,"MULTIPOLYGON (((143.00432 -33.78165, 143.03952..."
4,Bathurst Regional,"MULTIPOLYGON (((149.91212 -33.39582, 149.91144..."


,name,geometry
0,Capital Region,"MULTIPOLYGON (((150.05261 -37.26253, 150.05218..."
1,Central Coast,"MULTIPOLYGON (((151.31497 -33.55578, 151.31509..."
2,Central West,"MULTIPOLYGON (((150.14236 -32.34153, 150.14372..."
3,Coffs Harbour - Grafton,"MULTIPOLYGON (((153.07639 -30.42982, 153.07668..."
4,Far West and Orana,"MULTIPOLYGON (((148.67618 -29.50976, 148.67961..."


,name,geometry
0,New South Wales,"MULTIPOLYGON (((159.0623 -31.50886, 159.06218 ..."
1,Victoria,"MULTIPOLYGON (((146.29286 -39.15778, 146.29341..."
2,Queensland,"MULTIPOLYGON (((142.5314 -10.68301, 142.53072 ..."
3,South Australia,"MULTIPOLYGON (((140.66025 -38.06256, 140.66004..."
4,Western Australia,"MULTIPOLYGON (((117.86953 -35.19108, 117.86979..."


#### Step 3: Transform and Generate Rows for Each Level

This cell iterates through the different location levels (LGA, SA4, State, StateRemoteness), merges the attributes with the corresponding geometries loaded in the previous step, and prepares DataFrames for each level.

In [42]:
# --- 1. Preprocessing ---

# Handle potential explicit 'Unknown' strings or NaNs consistently.
# Replace specific string 'Unknown' with NaN for easier processing.
unknown_value = 'Unknown'
unique_locations_df = unique_locations_df.replace(unknown_value, np.nan)

# Preprocess State GDF: Map full names to abbreviations
if state_geom_gdf is not None and 'name' in state_geom_gdf.columns:
    state_geom_gdf['name'] = state_geom_gdf['name'].replace(state_mapping)
    logging.info("Mapped full state names to abbreviations in state_geom_gdf.")
else:
    logging.warning("state_geom_gdf is None or 'name' column missing. Cannot map state names.")

# Ensure geometry GDFs have the correct CRS based on DB TARGET
try:
    if lga_geom_gdf is not None and lga_geom_gdf.crs.to_epsg() != LGA_TARGET_SRID:
        logging.info(f"Reprojecting LGA geometries to EPSG:{LGA_TARGET_SRID}...")
        lga_geom_gdf = lga_geom_gdf.to_crs(f"EPSG:{LGA_TARGET_SRID}")

    if sa4_geom_gdf is not None and sa4_geom_gdf.crs.to_epsg() != SA4_STATE_TARGET_SRID:
        logging.info(f"Reprojecting SA4 geometries to EPSG:{SA4_STATE_TARGET_SRID}...")
        sa4_geom_gdf = sa4_geom_gdf.to_crs(f"EPSG:{SA4_STATE_TARGET_SRID}")

    if state_geom_gdf is not None and state_geom_gdf.crs.to_epsg() != SA4_STATE_TARGET_SRID:
        logging.info(f"Reprojecting State geometries to EPSG:{SA4_STATE_TARGET_SRID}...")
        state_geom_gdf = state_geom_gdf.to_crs(f"EPSG:{SA4_STATE_TARGET_SRID}")
except Exception as e:
    logging.error(f"Error setting/checking geometry CRS: {e}")
    # Decide how to proceed - stop or continue without correct CRS?

# Rename geometry columns in source GDFs for clarity during merge
if lga_geom_gdf is not None:
    lga_geom_gdf = lga_geom_gdf.rename(columns={'geometry': 'lga_geom'})
if sa4_geom_gdf is not None:
    sa4_geom_gdf = sa4_geom_gdf.rename(columns={'geometry': 'sa4_geom'})
if state_geom_gdf is not None:
    state_geom_gdf = state_geom_gdf.rename(columns={'geometry': 'state_geom'})


# --- 2. Determine Location Level ---

def determine_location_level(row):
    # Check from most specific to least specific
    if pd.notna(row['lga_name_2021']):
        return 'LGA'
    elif pd.notna(row['sa4_name_2021']):
        return 'SA4'
    elif pd.notna(row['state']):
        return 'State'
    else:
        return 'Unknown' # Should not happen if state is always present

unique_locations_df['location_level'] = unique_locations_df.apply(determine_location_level, axis=1)
logging.info("Determined location_level for each row.")
# Display value counts for verification
print("\nLocation Level Distribution:")
print(unique_locations_df['location_level'].value_counts())


# --- 3. Merge Geometries ---

# Start with the unique locations and levels
final_gdf = unique_locations_df.copy()

# Merge LGA geometries
if lga_geom_gdf is not None:
    logging.info(f"Merging LGA geometries (left join on 'lga_name_2021')...")
    final_gdf = pd.merge(
        final_gdf,
        lga_geom_gdf[['name', 'lga_geom']],
        how='left',
        left_on='lga_name_2021',
        right_on='name'
    )
    final_gdf = final_gdf.drop(columns=['name']) # Drop the merge key from gdf
else:
    logging.warning("LGA geometry GDF not available. Skipping LGA geometry merge.")
    final_gdf['lga_geom'] = None # Add empty column if GDF is missing

# Merge SA4 geometries
if sa4_geom_gdf is not None:
    logging.info(f"Merging SA4 geometries (left join on 'sa4_name_2021')...")
    final_gdf = pd.merge(
        final_gdf,
        sa4_geom_gdf[['name', 'sa4_geom']],
        how='left',
        left_on='sa4_name_2021',
        right_on='name'
    )
    final_gdf = final_gdf.drop(columns=['name']) # Drop the merge key from gdf
else:
    logging.warning("SA4 geometry GDF not available. Skipping SA4 geometry merge.")
    final_gdf['sa4_geom'] = None

# Merge State geometries
if state_geom_gdf is not None:
    logging.info(f"Merging State geometries (left join on 'state')...")
    # Ensure state column types match (should both be string)
    final_gdf['state'] = final_gdf['state'].astype(str)
    state_geom_gdf['name'] = state_geom_gdf['name'].astype(str)

    final_gdf = pd.merge(
        final_gdf,
        state_geom_gdf[['name', 'state_geom']],
        how='left',
        left_on='state',
        right_on='name'
    )
    final_gdf = final_gdf.drop(columns=['name']) # Drop the merge key from gdf
else:
    logging.warning("State geometry GDF not available. Skipping State geometry merge.")
    final_gdf['state_geom'] = None

logging.info("Finished merging geometries.")

# --- 4. Final Structure and Verification ---

# Define the final expected column order matching dim_location (excluding location_key)
all_columns = [
    "state",
    "sa4_name_2021",
    "lga_name_2021",
    "remoteness_area",
    "location_level",
    "lga_geom",
    "sa4_geom",
    "state_geom",
]

# Ensure all columns exist, adding missing ones as None if necessary
for col in all_columns:
    if col not in final_gdf.columns:
        final_gdf[col] = None
        logging.warning(f"Column '{col}' was missing after merges, added as None.")

# Reorder columns and fill NaNs in non-geometry columns before export
final_gdf = final_gdf[all_columns]
final_gdf['state'] = final_gdf['state'].fillna('Unknown') # Reinstate Unknown if needed
final_gdf['sa4_name_2021'] = final_gdf['sa4_name_2021'].fillna('Unknown')
final_gdf['lga_name_2021'] = final_gdf['lga_name_2021'].fillna('Unknown')
final_gdf['remoteness_area'] = final_gdf['remoteness_area'].fillna('Unknown')
final_gdf['location_level'] = final_gdf['location_level'].fillna('Unknown')


# Display info about merged geometries
print("\nGeometry Merge Summary:")
print(f" - LGA Geometries merged (not null): {final_gdf['lga_geom'].notna().sum()} / {len(final_gdf)}")
print(f" - SA4 Geometries merged (not null): {final_gdf['sa4_geom'].notna().sum()} / {len(final_gdf)}")
print(f" - State Geometries merged (not null): {final_gdf['state_geom'].notna().sum()} / {len(final_gdf)}")

# Display sample data
print("\nSample of final_gdf (before geometry conversion for CSV):")
print(final_gdf.head())

2025-04-09 22:47:10,119 - INFO - Mapped full state names to abbreviations in state_geom_gdf.
2025-04-09 22:47:10,126 - INFO - Determined location_level for each row.
2025-04-09 22:47:10,129 - INFO - Merging LGA geometries (left join on 'lga_name_2021')...
2025-04-09 22:47:10,135 - INFO - Merging SA4 geometries (left join on 'sa4_name_2021')...
2025-04-09 22:47:10,137 - INFO - Merging State geometries (left join on 'state')...
2025-04-09 22:47:10,141 - INFO - Finished merging geometries.



Location Level Distribution:
location_level
LGA      789
State     13
Name: count, dtype: int64

Geometry Merge Summary:
 - LGA Geometries merged (not null): 763 / 802
 - SA4 Geometries merged (not null): 788 / 802
 - State Geometries merged (not null): 802 / 802

Sample of final_gdf (before geometry conversion for CSV):
  state                           sa4_name_2021      lga_name_2021  \
0   NSW                                Riverina        Wagga Wagga   
1   NSW  Sydney - Baulkham Hills and Hawkesbury         Hawkesbury   
2   Tas               Launceston and North East  Northern Midlands   
3   NSW              New England and North West  Armidale Regional   
4   Vic                                 Unknown            Unknown   

            remoteness_area location_level  \
0  Inner Regional Australia            LGA   
1  Inner Regional Australia            LGA   
2  Inner Regional Australia            LGA   
3  Outer Regional Australia            LGA   
4                   Unkno

#### Step 4: Write to CSV File

Save the processed DataFrame to a CSV file.

In [62]:
# --- Convert Geometries to WKT with SRID Prefix ---
def geometry_to_ewkt(geom, srid):
    """Converts a Shapely geometry object to EWKT format (SRID=xxxx;WKT)."""
    if geom is None or pd.isna(geom):
        return '' # Return empty string for NULL in COPY

    # Ensure geometry is MultiPolygon if required by DB
    # Note: This might be computationally intensive if not already MultiPolygons
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    elif geom.geom_type != 'MultiPolygon':
        # Handle other types if necessary (e.g., GeometryCollection)
        # For simplicity, returning empty string if not Polygon/MultiPolygon
        # logging.warning(f"Unexpected geometry type {geom.geom_type}, exporting as NULL.")
        return ''

    # Use shapely.wkt.dumps for robust WKT conversion
    wkt_string = wkt_dumps(geom, rounding_precision=10)  # Adjust precision as needed
    return f"SRID={srid};{wkt_string}"

logging.info("Converting geometry columns to EWKT format...")
final_gdf['lga_geom_wkt'] = final_gdf['lga_geom'].apply(lambda x: geometry_to_ewkt(x, LGA_TARGET_SRID))
final_gdf['sa4_geom_wkt'] = final_gdf['sa4_geom'].apply(lambda x: geometry_to_ewkt(x, SA4_STATE_TARGET_SRID))
final_gdf['state_geom_wkt'] = final_gdf['state_geom'].apply(lambda x: geometry_to_ewkt(x, SA4_STATE_TARGET_SRID))
logging.info("Geometry conversion complete.")

# --- Select and Order Columns for CSV ---
# Must match the order expected by COPY (excluding the auto-generated location_key)
# Replace original geometry columns with their WKT versions
csv_columns = [
    "state",
    "sa4_name_2021",
    "lga_name_2021",
    "remoteness_area",
    "location_level",
    "lga_geom_wkt", # Use WKT column
    "sa4_geom_wkt", # Use WKT column
    "state_geom_wkt", # Use WKT column
]
export_df_final = final_gdf[csv_columns]

# --- Check VARCHAR Lengths (Optional but Recommended) ---
varchar_limits = {
    "state": 7,
    "sa4_name_2021": 50,
    "lga_name_2021": 50,
    "remoteness_area": 50,
    "location_level": 20
}
for col, limit in varchar_limits.items():
    max_len = export_df_final[col].astype(str).str.len().max()
    if max_len > limit:
        logging.warning(f"WARNING: Max length for column '{col}' ({max_len}) exceeds DB schema limit ({limit}). Data might be truncated on import.")


# --- Export to CSV ---
logging.info(f"Exporting data to {OUTPUT_CSV_FILE}...")
export_df_final.to_csv(
    OUTPUT_CSV_FILE,
    index=False,
)
logging.info(f"Successfully exported {len(export_df_final)} rows to {OUTPUT_CSV_FILE}.")

2025-04-09 23:23:46,487 - INFO - Converting geometry columns to EWKT format...


2025-04-09 23:24:28,726 - INFO - Geometry conversion complete.
2025-04-09 23:24:28,762 - INFO - Exporting data to output/dim_location.csv...
2025-04-09 23:24:47,784 - INFO - Successfully exported 802 rows to output/dim_location.csv.


### dim_population_lga

In [63]:
year_cols = [str(y) for y in range(2001, 2024)]

# Melt the raw population data
pop_lga_melted = pd.melt(
    pop_LGA_df,
    id_vars=["lga_name_2021"],
    value_vars=year_cols,
    var_name="year",
    value_name="population",
)

# Basic cleaning and type conversion
pop_lga_melted["year"] = pop_lga_melted["year"].astype(int)
pop_lga_melted["population"] = pd.to_numeric(
    pop_lga_melted["population"], errors="coerce"
)
pop_lga_melted.dropna(subset=["population"], inplace=True)
pop_lga_melted["population"] = pop_lga_melted["population"].astype(int)

# Merge melted population data with the location dimension data
# This adds the 'location_key' based on matching 'lga_name_2021'.
# 'how="inner"' ensures only LGAs present in both DataFrames are kept.
dim_location_keyed = final_gdf.copy()
dim_location_keyed['location_key'] = range(1, len(dim_location_keyed) + 1)

pop_lga_melted = pd.merge(
    pop_lga_melted,
    dim_location_keyed[['location_key', 'lga_name_2021']],
    how='inner',
    on='lga_name_2021'
)

In [65]:
# Select and reorder the final columns according to the target table structure
dim_population_lga_df = pop_lga_melted[["location_key", "year", "population"]]

dim_population_lga_df.to_csv("output/dim_population_lga.csv", index=False)

### dim_population_remoteness

In [69]:
# Assign basic column names
pop_remoteness_df.columns = ["state_raw", "remoteness_raw"] + year_cols

# Forward fill the State column
pop_remoteness_df["state_raw"] = pop_remoteness_df["state_raw"].ffill()

# Melt
pop_remote_melted = pd.melt(
    pop_remoteness_df,
    id_vars=["state_raw", "remoteness_raw"],
    value_vars=year_cols,
    var_name="year",
    value_name="population",
)

# Basic cleaning and type conversion
pop_remote_melted["year"] = pop_remote_melted["year"].astype(int)
pop_remote_melted["population"] = pd.to_numeric(
    pop_remote_melted["population"], errors="coerce"
)
pop_remote_melted.dropna(
    subset=["population", "remoteness_raw"], inplace=True
)  # Drop rows with no population or remoteness name
pop_remote_melted["population"] = pop_remote_melted["population"].astype(int)

# Map state names
pop_remote_melted["state"] = (
    pop_remote_melted["state_raw"].map(state_mapping).fillna("Unknown")
)

# Basic Remoteness name cleaning (remove state suffix)
pop_remote_melted["remoteness_area"] = (
    pop_remote_melted["remoteness_raw"]
    .astype(str)
    .str.replace(r"\s*\(.*\)$", "", regex=True)
    .str.strip()
)

# Filter out unknown states and potentially totals/migratory rows if they cause issues
pop_remote_melted = pop_remote_melted[pop_remote_melted["state"] != "Unknown"]

pop_remote_melted = pd.merge(
    pop_remote_melted,
    dim_location_keyed[["location_key", "state", "remoteness_area"]],
    how="inner",
    on=["state", "remoteness_area"],
)

In [70]:
# Select final columns
dim_population_remoteness_df = pop_remote_melted[["location_key", "year", "population"]]
dim_population_remoteness_df.to_csv("output/dim_population_remoteness.csv", index=False)

### dim_dwelling

In [ ]:
dwellings_df.columns = ["lga_name_2021", "dwelling_count"]

dwellings_df = pd.merge(
    dwellings_df,
    dim_location_keyed[["location_key", "lga_name_2021"]],
    how="inner",
    on="lga_name_2021",
)

dwellings_csv = dwellings_df[["location_key", "dwelling_count"]]

dwellings_csv.to_csv("output/dim_dwelling.csv", index=False)

## Create Fact Table

### fact_fatality

In [ ]:
logging.info("Starting Fact Table Generation...")

# --- Step 1: Add Surrogate Keys to Dimension DataFrames (Simulating DB Load) ---

logging.info("Adding surrogate keys to dimension DataFrames...")

# Make copies to avoid modifying originals if needed elsewhere
dim_date_keyed = dim_date_df.copy()
dim_timeofday_keyed = dim_timeofday_df.copy()
dim_roaduser_keyed = dim_roaduser_df.copy()
dim_crash_keyed = dim_crash_df.copy()
# Use the location DF BEFORE geometry was converted to WKT for merging
dim_location_keyed = final_gdf.copy()

# Add keys (starting from 1 like SERIAL/BIGSERIAL)
dim_date_keyed['date_key'] = range(1, len(dim_date_keyed) + 1)
dim_timeofday_keyed['time_key'] = range(1, len(dim_timeofday_keyed) + 1)
dim_roaduser_keyed['road_user_key'] = range(1, len(dim_roaduser_keyed) + 1)
dim_crash_keyed['crash_key'] = range(1, len(dim_crash_keyed) + 1)
dim_location_keyed['location_key'] = range(1, len(dim_location_keyed) + 1)

logging.info("Surrogate keys added.")

# Display sample of keyed dimensions (optional check)
# print("Dim Date with Key (Head):")
# display(dim_date_keyed.head())
# print("\nDim Location with Key (Head):")
# display(dim_location_keyed.head())

# --- Step 2: Prepare fact_df for Merging ---
# Ensure data types and values used for merging match how dimensions were created

logging.info("Preparing fact_df for merging with dimension keys...")

# Create a working copy
fact_merged_df = fact_df.copy()

# Time: Convert 'Time_victim' to datetime.time object for merging with dim_timeofday
fact_merged_df['time_for_merge'] = pd.to_datetime(
    fact_merged_df['Time_victim'], format='%H:%M:%S', errors='coerce'
).dt.time

# Hour_24: Re-calculate hour based on the cleaned time for merging consistency
fact_merged_df['hour_24_for_merge'] = pd.to_datetime(
    fact_merged_df['Time_victim'].astype(str), format='%H:%M:%S', errors='coerce'
).dt.hour.astype('Int64') # Use Int64 to allow NA

# Road User: Ensure 'Unknown' fill matches dim_roaduser creation
fact_merged_df['Age Group'] = fact_merged_df['Age Group'].fillna("Unknown")
# Rename columns to match dim_roaduser_keyed
fact_merged_df.rename(
    columns={
        "Road User": "road_user_type",
        "Gender": "gender",
        "Age": "age",
        "Age Group": "age_group",
        "State_victim": "state_orig", # Keep original names for location merge
        "SA4 Name 2021_victim": "sa4_name_2021_orig",
        "National LGA Name 2021_victim": "lga_name_2021_orig",
        "National Remoteness Areas_victim": "remoteness_area_orig",
        "Time of day_victim": "time_of_day_category", # Match dim_timeofday
        "Crash ID": "crash_id", # Match dim_crash
    },
    inplace=True,
)
# Handle potential NA's in Age before merge (convert to a type that handles NA, like float or Int64)
fact_merged_df['age'] = fact_merged_df['age'].astype('Int64')

# Location: Apply the same fillna logic used for creating dim_location
location_cols_orig = ["state_orig", "sa4_name_2021_orig", "lga_name_2021_orig", "remoteness_area_orig"]
for col in location_cols_orig:
    fact_merged_df[col] = fact_merged_df[col].fillna("Unknown")


# Crash: Ensure 'Unknown' fill matches dim_crash creation for relevant columns
# (Only crash_id is needed for the merge key itself, but good practice if checking other attrs)
# fact_merged_df['crash_type'] = fact_merged_df['Crash Type_victim'].fillna("Unknown") # Example if needed

logging.info("fact_df prepared for merging.")

# --- Step 3: Merge Dimension Keys into fact_merged_df ---

logging.info("Merging dimension keys into fact data...")

# Merge Date Key
fact_merged_df = pd.merge(
    fact_merged_df,
    dim_date_keyed[['year', 'month', 'christmas_period', 'easter_period', 'date_key']],
    how='left',
    left_on=['Year_victim', 'Month_victim', 'Christmas Period_victim', 'Easter Period_victim'],
    right_on=['year', 'month', 'christmas_period', 'easter_period']
)

# Merge Time Key
fact_merged_df = pd.merge(
    fact_merged_df,
    dim_timeofday_keyed[['time', 'time_of_day_category','hour_24', 'time_key']],
    how='left',
    left_on=['time_for_merge', 'time_of_day_category', 'hour_24_for_merge'],
    right_on=['time', 'time_of_day_category', 'hour_24']
)

# Merge Road User Key
fact_merged_df = pd.merge(
    fact_merged_df,
    dim_roaduser_keyed[['road_user_type', 'gender', 'age', 'age_group', 'road_user_key']],
    how='left',
    on=['road_user_type', 'gender', 'age', 'age_group'] # Use already renamed columns
)

# Merge Crash Key
fact_merged_df = pd.merge(
    fact_merged_df,
    dim_crash_keyed[['crash_id', 'crash_key']],
    how='left',
    on='crash_id' # Use already renamed column
)

# Merge Location Key
# Use the original location columns from fact_df (after fillna) and merge with dim_location_keyed
fact_merged_df = pd.merge(
    fact_merged_df,
    dim_location_keyed[['state', 'sa4_name_2021', 'lga_name_2021', 'remoteness_area', 'location_level', 'location_key']],
    how='left',
    left_on=['state_orig', 'sa4_name_2021_orig', 'lga_name_2021_orig', 'remoteness_area_orig'],
    right_on=['state', 'sa4_name_2021', 'lga_name_2021', 'remoteness_area']
)

logging.info("Finished merging dimension keys.")

# --- Step 4: Check for Missing Keys ---
missing_keys = fact_merged_df[['date_key', 'time_key', 'location_key', 'crash_key', 'road_user_key']].isnull().sum()
logging.info(f"Checking for missing keys after merge:\n{missing_keys}")

if missing_keys.sum() > 0:
    logging.warning("Missing keys found! Rows with missing keys will be dropped.")
    # Optional: Inspect rows with missing keys
    # display(fact_merged_df[fact_merged_df['location_key'].isnull()].head())
    fact_merged_df.dropna(
        subset=['date_key', 'time_key', 'location_key', 'crash_key', 'road_user_key'],
        inplace=True
    )
    # Convert keys to integer after dropping NaNs
    key_cols = ['date_key', 'time_key', 'location_key', 'crash_key', 'road_user_key']
    fact_merged_df[key_cols] = fact_merged_df[key_cols].astype(int)


# --- Step 5: Create fact_fatality_df ---
logging.info("Creating fact_fatality DataFrame...")

# Add the measure
fact_merged_df['fatality_count'] = 1

# Select and order columns for the final fact table
fact_fatality_df = fact_merged_df[[
    'date_key',
    'time_key',
    'location_key',
    'crash_key',
    'road_user_key',
    'fatality_count' # Measure
]].copy()

# Optional: Add primary key (though DB handles this)
# fact_fatality_df.insert(0, 'fatality_key', range(1, len(fact_fatality_df) + 1))

logging.info(f"Created fact_fatality_df with {len(fact_fatality_df)} rows.")
display(fact_fatality_df.head())

fact_fatality_df.to_csv("output/fact_fatality.csv", index=False)
logging.info(f"Successfully exported {len(fact_fatality_df)} rows to fact_fatality.csv.")


2025-04-09 19:51:18,267 - INFO - Starting Fact Table Generation...
2025-04-09 19:51:18,268 - INFO - Adding surrogate keys to dimension DataFrames...
2025-04-09 19:51:18,318 - INFO - Surrogate keys added.
2025-04-09 19:51:18,319 - INFO - Preparing fact_df for merging with dimension keys...
2025-04-09 19:51:18,425 - INFO - fact_df prepared for merging.
2025-04-09 19:51:18,425 - INFO - Merging dimension keys into fact data...
2025-04-09 19:51:18,629 - INFO - Finished merging dimension keys.
2025-04-09 19:51:18,631 - INFO - Checking for missing keys after merge:
date_key          0
time_key          0
location_key      0
crash_key         0
road_user_key    34
dtype: int64
2025-04-09 19:51:18,632 - WARNING - Missing keys found! Rows with missing keys will be dropped.
2025-04-09 19:51:18,654 - INFO - Creating fact_fatality DataFrame...
2025-04-09 19:51:18,662 - INFO - Created fact_fatality_df with 56787 rows.


,date_key,time_key,location_key,crash_key,road_user_key,fatality_count
0,1,1,1,1,1,1
1,2,2,2,2,2,1
2,1,3,3,3,3,1
3,2,4,4,4,4,1
4,2,5,5,5,5,1


2025-04-09 19:51:18,717 - INFO - Successfully exported 56787 rows to fact_fatality.csv.
